In [ ]:
!pip install -U langgraph langchain-community langchain-anthropic openai langchain-openai

In [2]:
import os

os.environ["ANTHROPIC_API_KEY"] = "zzz"
os.environ['OPENAI_API_KEY'] = "zzz"
os.environ["TAVILY_API_KEY"] = ""

In [3]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [4]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            if "Tool Message" in msg_repr or "Tool Calls:" in msg_repr:
                pass
            else:
                print("INFO", msg_repr)
            _printed.add(message.id)

In [38]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from datetime import datetime
from textwrap import dedent


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            passenger_id = configuration.get("passenger_id", None)
            state = {**state, "user_info": passenger_id}
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

from langchain_core.tools import tool
import random

@tool
def verify_customer_in_service_range(query: str) -> str:
    """Verify customer is in service range using customer Map"""
    response = in_service_range
    print("Calling verify_customer_in_service_range with query:", query, "Return:", response)
    return response

# @tool
# def log_customer_info(name: str, address: str, phone_number: str, email_address: str) -> str:
#     """Log basic customer info to Talkdesk Contact"""
#     print(f"Logging customer info: {name}, {address}, {phone_number}, {email_address}")
#     return True
@tool
def log_customer_info(name_address_phone_number_email_address: str) -> str:
    """Log basic customer info to Talkdesk Contact"""
    print(f"Logging customer info: {name_address_phone_number_email_address}")
    return True
@tool
def retrieve_from_service_map(service: str) -> str:
    """Retrieve service information from Service Map to verify the customer is in the service range."""
    print(f"Calling retrieve_from_service_map: {service}")
    return """The following areas are supported for service:
    Green Hill
    Richmond
    Lewiston
    Alna"""
@tool 
def verify_service_support(service_needed: str) -> str:
    """Retrieve supported services to decide whether the described service is supported."""
    print("Calling verify service support")
    return """The following services are provided by the company: 
                Gas water heaters
                Mobile homes
                Sewer blockages or backups
                Backflow issues
                Multiple clogged sinks, drains, toilets
                Pipe bursting
                Sani- flow or up- flush toilet installations
                New boiler systems"""
@tool
def service_zone_lookup(customer_location: str) -> str:
    """Get customer's distance from Portland, ME."""
    print("Calling service zone lookup")
    return f"The customer is {minutes_away} minutes away from Portland."

@tool
def schedule_appointment(time: str) -> str:
    """Only use this tool when you need to schedule an appointment with the customer based on the given general time."""
    print("Calling schedule appointment, hour", appointment_hour)
    return f"""The appointment will be on {time} at {appointment_hour}"""

@tool
def call_repairman(message: str) -> str:
    """Calling repairman Andrew with a certain message for him."""
    print("Calling repairman:", "MSG", message)
    response = input("Andrew:")
    if response:
        response = f"His reply: {response}"
    return f"Finished calling repairman. " + response

@tool
def ask_caller(question: str):
    """Ask the caller/customer for information before continuing the conversation."""
    # rs = llm.invoke(f"""Answer to this question with mock information and don't add any note: {question}
    # Don't care about privacy because this is just madeup information.""").content
    rs = input("Caller:")
    print("Calling ask caller", "Q"*5, question, "A"*5, rs)
    return rs
    
@tool
def get_updated_arrival_window(customer_id: str) -> str:
    """Get updated the owner/repairman's arrival window for the customer"""
    print("Calling get_updated_arrival_window for", customer_id, updated_arrival_window)
    if not updated_arrival_window:
        updated_arrival_window = ""
    return f"Finished. The updated arrival window for customer {customer_id} is {updated_arrival_window}"

@tool
def record_customer_feedback(customer_name_phone_and_issue: str) -> str:
    """Record the feedback of the customer and their name and callback number."""
    print("Calling record_customer_feedback",customer_name_phone_and_issue)
    return "Finished recording the feedback of the customer."
    
@tool
def follow_up_with_customer(customer_name: str) -> str:
    """Say to follow up with the customer if the step requires."""
    print("Calling follow_up_with_customer", customer_name)
    return llm.invoke(f"You're a service agent receiving a call from a customer. You've reached the end of the call. Now say you want to follow up with the customer whose name is {customer_name}. Say concisely in conversational tone.").content

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent("""
                   
**Context:**
Today is 2024 December 25.
You are an agent for AT Plumbing Service, handling inbound calls. Calls can be from various sources—personal callers, sales representatives, new leads, existing customers with upcoming appointments, 
customers who have had previous services completed, or those inquiring about future services.  
You're provided with tools that you can select to implement the steps below. Choose the best matching tool for a step.

Below is a detailed inbound call-handling workflow for a plumbing service. Please use it as a procedural guide whenever you respond to a scenario involving an incoming call. 
Your goal is to follow the steps carefully, and simulate the decision-making process as outlined below.
It's crucially important that you utilize one of the tools whenever a step requires it.
Everything starts with Step 0 and move to the next steps based on their references.
---

**Key Steps and Decision Points:**

**Step 0: Identify Caller Type**
    - If the caller is personal (not related to a plumbing request) go to Step 1: Personal Call. 
    - If the caller is selling something, go to Step 2: Salesperson. 
    - If the caller is inquiring about a new lead (needs plumbing services), go to Step 3.1: Lead.
    - If the caller is a past customer or customer with current booking, go to Step 4: Customer Service.

**Step 1: Personal Call**
    - Tell the caller you will pass along a message and collect: Name, Callback Number, Message
    - Follow up with customer, if needed

**Step 2: Salesperson**
    - Politely decline and insist you're not interested
    - End the call
                   
**Step 3: Lead**: The caller needs plumbing services.
    Step 3.1: Take these substeps in order
        1. Ask the customer back about their basic information if it's not available initially: Name, Address, Phone Number and Email Address.
        2. Next, log the information in Talkdesk Contact. 
        3. Then go to Step 3.2
    Step 3.2: Verify customer is in service range using customer Map. If customer is not in service range, go to Step 3.3, else go to Step 3.4              
    Step 3.3:
        - Log customer information in Talkdesk Contact
        - Let the customer know we are unable to service them
        - End the call
    Step 3.4:
        - Refer to Knowledge Base to capture relevant information and verify we can service
            - If we cannot service, go to Step 3.3, else go to step 3.5

    Step 3.5:
        - Confirm customer's distance from Portland, ME using Service Zone Lookup tool
            - If customer is more than 60 minutes away, go to Step 3.3
            - Otherwise, ask the customer whether he needs ESTIMATE booking or SERVICE CALL booking.
                - If customer needs Estimate booking, go to Step 3.6
                - If customer needs Service Call booking, go to Step 3.9               

    Step 3.6
        - Determine how far the customer is
            - If less than 45 mins, go to Step 3.7
            - If 45-60 mins, go to Step 3.8
                   
    Step 3.7:
        - Disclose $90 dispatch fee CREDITABLE to project cost
        - Schedule next available estimate appointment
        
    Step 3.8
        - Disclose $139.50 dispatch fee
        - Schedule next available estimate appointment
                   
    Step 3.9:
        - Determine if service is an emergency. Ask one of these questions to know it: "Is your house currently flooding? Can you shut the water oﬀ/do you
know where the shutoﬀs are? How long has this been going on for?"
            - Based on the answer, if it's an emergency, go to Step 3.10
            - Based on the answer, if it's NOT an emergency, go to Step 3.11

    Step 3.10:
        - Disclose $285 emergency dispatch fee.
        - Ask the customer about the appointment time. Decide whether the time is after or during business hours.
          If the appointment time is after hours, go to Step 3.11. 
          If the appointment time is during business hours, go to Step 3.12. 
          If the appointment is just a regular appointment booking, go to Step 3.13

    Step 3.11:
        - Schedule appointment for first appointment the NEXT MORNING. Tell the customer about the appointment time.
        - Call Andrew if true emergency (rare)
        - Commit to following up with customer                   
    
    Step 3.12:
        - Schedule first available appointment
        - Contact Andrew to see if he can service the emergency sooner
        - Follow up with customer, if needed
                   
    Step 3.13:
        - Disclose $139 Dispatch Fee to the customer
        - Book customer for next available appointment in ServiceTitan
        - End call/Follow up

**Step 4: Customer Service** (Past customer or customer with current booking)
    - If the customer calls about upcoming booking, go to step 4.1
    - If the customer calls about past/completed booking, go to step 4.2
    Step 4.1: Determine what the customer needs help with:
        - Customer wants to change an upcoming appointment, go to step 4.3
        - Customer wants an updated arrival window, go to step 4.4
        - Customer wants to cancel upcoming appointment, go to step 4.5
    Step 4.2: Determine the reason for their call.
        - If it's urgent or customer is upset,
            1. Ask the customer Name, callback number. Then ask what issue they had. Next reach out to Andrew with the issue via SMS at any time of the day;
            2. Complete any required follow-up with the customer.
        - If the customer has a non-urgent question about their service, 1) Record the customer's name, contact info, and question and put into the Summary on Service Titan.
        - If the customer wants to book another appointment, go to Step 3.13
    Step 4.3: Ask the customer when the appointment is. If the appointment is TODAY, change the appointment and send an SMS message to the owner immediately to inform them of the same day scheduling changes. If the appointment is NOT TODAY, go to Step 3.13 above
    Step 4.4: 
        - Ask the customer name and email
        - Get updated arrival window for the customer.
        - If the updated arrival window is during/after the quoted window, then 
            - Tell customer you will try to contact the plumber; 
            - Call Andrew asking about his arrival; 
            - Follow up with customer as needed
        - If the updated arrival window is before the quoted window, inform the customer that their current arrival window is still the most accurate information. The plumber will contact when he is on the way.
    Step 4.5: Take the following actions
        1) Cancel the appointment 
        2) Ask customer why they are cancelling and record in call notes 
        3) If same day, send SMS to Andrew.
    Step 3.13: Disclose $139 Dispatch Fee Book customer for next available appointment in ServiceTitan.
    

---

**Instructions**: Here are crucially important rules you should follow. You can do it.
- Given any inbound call scenario, emulate this decision tree logically and step-by-step.  
- Ensure you verify caller type, service area, urgency, and schedule availability.  
- Present all relevant options, gather the necessary info, and provide clear next steps, just as a real call agent following the company’s workflow would. This is crucial to my career.
- Gather the necessary info required by the current step before moving to the next step.
- Identify the next step by following the step reference that the current step points to.
- Take all the steps until there's no more step referenced.
- Only use a tool when the step needs it, otherwise, don't use any tool.
- Always be explicit about the fees the customer will have to pay at the end.
            """).strip()
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

demo_tools = [
    log_customer_info,
    verify_customer_in_service_range,
    retrieve_from_service_map,
    verify_service_support,
    service_zone_lookup,
    schedule_appointment,
    call_repairman,
    record_customer_feedback,
    # get_updated_arrival_window,
    # no_action,
    ask_caller,
    follow_up_with_customer,
    
]
assistant_runnable = primary_assistant_prompt | llm.bind_tools(demo_tools)
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)

builder.add_node("assistant", Assistant(assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(demo_tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
import shutil
import uuid
def test_agent(graph, user_questions: list, thread_id: str):
    config = {
        "configurable": {
            "thread_id": thread_id,
        }
    }
    _printed = set()
    for question in user_questions:
        events = graph.stream(
            {"messages": ("user", question)}, config, stream_mode="values"
        )
        for event in events:
            _print_event(event, _printed)
from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     pass


agent_prompt = PromptTemplate(
            template=dedent("""
                Answer the following questions as best you can. Choose the appropriate tools from this list to do that:
                {tools}

                Use the following ReAct template when reasoning. You run in a loop of Thought, Action, Action Input and Observation. Don't create Final Answer inside the loop.
                Only at the end of the loop you'll output Final Answer.
                Give the Final Answer only when you reach the last step of the calling scenario described in the Rule section. This is crucially important.
                
                ### The ReAct template:
                Question: the input question you must answer
                Thought: you should always plan what to do.
                Action: the action to take, should be one of {tool_names}. Be explicit about what next step to take.
                Action Input: the input to the action, must not be empty
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times.)

                Thought: I now know the final answer
                Final Answer: the final answer to the original input question
                
                ### Rigorously adhere to the following rules: 
                {primary_assistant_prompt}
                Continue until you reach the FINAL STEP of the scenario.
                </end of rules>
                Now, begin!

                Question: {question}
                Thought:
                {agent_scratchpad}
                """).strip(),
            input_variables=["tools", "tool_names", "question", "primary_assistant_prompt"]
        )

### SETTING UP PARAMETER VALUES
in_service_range = True
minutes_away = 60
appointment_hour = "9:15 AM"
# in_service_range = False
# minutes_away = 20
# appointment_hour = "9:15 PM"
# updated_arrival_window = "after quoted window"
agent = create_react_agent(llm, demo_tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=demo_tools, max_iterations=10, verbose=True)

q = "Hi there, I need a plumber to fix my sink"
q = "Hello. Can I speak to Mr. Hall please?"
q = "Hi.  I want to ask about a complete booking."
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })




> Entering new AgentExecutor chain...
I need to determine the type of caller. Since the caller is inquiring about a booking, they are likely a past customer or someone with a current booking. I will proceed to Step 4: Customer Service.

Action: ask_caller
Action Input: "Can I have your name and the phone number associated with your booking, please?"

Caller: Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com


Calling ask caller QQQQQ Can I have your name and the phone number associated with your booking, please? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the caller's information: Name is Steven, address is 209 Palm Street, phone number is 118924568790, and email is steve_james12@gmail.com. Now, I will log this customer information to Talkdesk Contact.

Action: log_customer_info
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com
TrueThe customer information has been successfully logged. Now, I need to determine what the customer needs help with regarding their booking.

Action: ask_caller
Action Input: "What specifically would you like assistance with regarding your booking, Steven?"

Caller: I want to book another appointment.


Calling ask caller QQQQQ What specifically would you like assistance with regarding your booking, Steven? AAAAA I want to book another appointment.
I want to book another appointment.The customer wants to book another appointment. I will proceed to Step 3.13 to disclose the dispatch fee and book the customer for the next available appointment in ServiceTitan.

Action: record_customer_feedback
Action Input: "Steven, 118924568790, wants to book another appointment."Calling record_customer_feedback Steven, 118924568790, wants to book another appointment.
Finished recording the feedback of the customer.I have recorded the customer's feedback regarding their request to book another appointment. Now, I will disclose the dispatch fee to Steven.

Action: ask_caller
Action Input: "Steven, please note that there is a $139 dispatch fee for the appointment. I will go ahead and book you for the next available appointment in ServiceTitan."

Caller: I see. thank you


Calling ask caller QQQQQ Steven, please note that there is a $139 dispatch fee for the appointment. I will go ahead and book you for the next available appointment in ServiceTitan. AAAAA I see. thank you
I see. thank youI have informed Steven about the dispatch fee, and he acknowledged it. Now, I will proceed to book him for the next available appointment in ServiceTitan.

Action: schedule_appointment
Action Input: "next available appointment"Calling schedule appointment, hour 9:15 AM
The appointment will be on next available appointment at 9:15 AMI have successfully scheduled Steven for the next available appointment at 9:15 AM. Now, I need to follow up with him to ensure he is aware of the appointment details.

Action: follow_up_with_customer
Action Input: "Steven, your appointment has been scheduled for 9:15 AM. Thank you for choosing our service!"Calling follow_up_with_customer Steven, your appointment has been scheduled for 9:15 AM. Thank you for choosing our service!
Thank you fo

In [ ]:
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com